*  This program computes and evaluates 
*                   the Black-Litterman asset allocation problem of
*                  1 riskfree asset with n>1 rsiky assets 
*                          Here we use 5 stocks and assume they are all the stocks in the mkt

In [15]:
import pandas as pd                     # To load data, we use the package pandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load the data 

df = pd.read_excel('Stocks.xlsx')          # return on 5 stocks, their prices and shares outstanding
R1 = df.loc[:,['AXP','HO','JNJ','MSFT','WMT']]         #  extract the 5 returns, R1 is T by 5.
Share = df.loc[:,['AXPS','HOS','JNJS','MSFTS','WMTS']]   # the shares outstanding
Price = df.loc[:,['AXPP','HOP','JNJP','MSFTP','WMTP']]        # the prices

df1 = pd.read_excel('FF_Fac_Jan90_Dec04.xlsx')        # It has 5 columns:  date, mkt, size, b/m, riskree rate
                                                     # downloaded from Ken French's website
rf = df1.loc[:,"rate"]/100                           # as the data are in percentage ponits

T = 180

R1 = np.array(R1)                         # convert list to array to apply np.functions
Share = np.array(Share) 
Price = np.array(Price) 
rf = np.array(rf)                          

T = len(df)                               # The number of obs

Re = np.ones((T,5))                           # creat storage for excess returns

for i in range(5):
    Re[:,i] = R1[:,i] - rf                # the excess return:  each substracts riskfree rate, Re[:,i]-rf

#  The mean-variance optimnal portfolio

mu = np.mean(Re, axis = 0)              # the mean taking each column of the matrix, a row vector of 5
mu = mu.T                                # make it a column vector
V = np.cov(Re.T)                        # the covariance estimate, 5 by 5 

VI = np.linalg.inv(V)                     # The inverse of V

# The optimal weights on the 5 risky aasets

gamma = 3                             # The risk-averse coeff.

w0 = 1/gamma*np.matmul(VI, mu)


#  The value-weighte portfolio

N = 5
value = np.ones((N,1))
Total = 0
for i in range (N):
    value[i] = Price[T-1,i]*Share[T-1,i]        #  mkt value of each firm
    Total = Total + value[i]
    
wv = value / Total

print(' The m-v optimal wights  \n')
print(w0) 
print('   \n')
print(' The value weights \n')
print(wv) 

 The m-v optimal wights  

[0.06400505 0.48888157 0.5896034  0.4527425  0.03397834]
   

 The value weights 

[[0.08162572]
 [0.108275  ]
 [0.21713021]
 [0.33491638]
 [0.25805269]]


In [34]:
# Consider now how to express views in the Black-Litterman framework

kappa = 0.1         # the precision paramter indicating the degree
                    # of belief in how the true expected return is
                    #  close to its equilibrium value Pai
      
       
K = 2            # Suppose we have K=2 views    
 
P = np.ones((2,N))         #  to store a 2 by N matrix to express our views
mu0 = np.ones((2,1))       #   the difference of our views between assets
Omega = np.zeros((2,2))      #   2 by 2 precision matrix of our views
                             # important: no longer need to set off diagnoal elements to zeros again
    
P[0,:] = np.array([1, -1, 0, 0, 0])       # set values for the first row; remebre index starts fr 0
mu0[0] = 0.01              
Omega[0,0] = 0.005*0.005        # With these values, the expected return of the 1st asset minus the 2nd is 0.01 with
                                # precision in terms of standard error being 0.005
      
        
P[1,:] = np.array([0, 0, 0, 1, -1])        # set values for the first row  
mu0[1] = 0.02          
Omega[1,1] = 0.01*0.010      # With these values, the expected return of the 4th asset minus the last is 0.01 with
                              #  precision in terms of standard error being 0.01
      
 
      #   update the parameters by the Black-Litterman model 

tau = gamma
Pai = tau*np.matmul(V, wv)                                # The implied returns of the assets  

OmegaI = np.linalg.inv(Omega)                     # The inverse of Omega
PO = np.matmul(P.T,OmegaI)                    # P'*inv(Omega) 
A = np.matmul(PO,P)+VI/kappa
AI = np.linalg.inv(A)                          # ( P'*inv(Omega)*P+inv(kappa*V) )^(-1)
B = np.matmul(PO,mu0) + np.matmul(VI,Pai) / kappa   # P'*inv(Omega)*mu0+inv(kappa*V)*Pai
 
muNew = np.matmul(AI, B)
Theta = AI 
Vnew = V + AI

VnewI = np.linalg.inv(Vnew)                     # The inverse of Omega
wBL = (1/tau)*np.matmul(VnewI, muNew)          # The portfolio weights based on the Black-Literman model

 
print(' The m-v optimal wights  \n')
print(w0) 
print('   \n')
print(' The value weights \n')
print(wv.T)     
print('   \n')
print(' The Black-Litterman weights \n')
print(wBL.T)    

 The m-v optimal wights  

[0.06400505 0.48888157 0.5896034  0.4527425  0.03397834]
   

 The value weights 

[[0.08162572 0.108275   0.21713021 0.33491638 0.25805269]]
   

 The Black-Litterman weights 

[[ 0.48719022 -0.3145532   0.1973911   0.62817019 -0.08910739]]
